Download Price Paid Data and Load into DB

In [1]:
# Verifies that CSV file is up-to-date (else: D/L from Land Registry server)

from ppc_db import fetch_csv

url = 'http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv'
dest = 'C:/Users/jamie/OneDrive/Python/Py_24/Docker/Data/'

file_name = fetch_csv(url,dest)

100%|██████████| 5.17G/5.17G [05:59<00:00, 14.4MB/s]


In [ ]:
# Copy local file into Docker volume

from ppc_db import copy_file_to_docker_volume

file_name = 'Oct24pricepaidcomplete'
mount_path = "/var/lib/postgresql/data"

copy_file_to_docker_volume(file_name+".csv",
                           container_name = "788303b91078",
                           local_path = "/Users/jamiegleave/Desktop/HP_Dat/",
                           mount_path = mount_path
                           )

In [1]:
# CREATE TABLE (SQL)

from ppc_db import postgresql_commitchanges

file_name = 'Oct24pricepaidcomplete'

db_args = {
    'host':'localhost',
    'database':'LandRegData',
    'user':'root',
    'password':'password'
 }

base = 'CREATE TABLE '+file_name+' '

fd = open('tbl_params.sql','r')
sqlFile = fd.read()
fd.close()

postgresql_commitchanges(query=base+sqlFile,**db_args)

In [2]:
# LOAD DATA (SQL)

mount_path = "/var/lib/postgresql/data"

start = "COPY "+file_name+" FROM '"+mount_path+"/"+file_name+".csv'"
end = """ WITH (FORMAT csv, HEADER false, DELIMITER ',', QUOTE '"', ESCAPE '"')"""

postgresql_commitchanges(start+end, **db_args)